In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# from google.colab import drive
# drive.mount('/content/drive/')

# %cd /content/drive/MyDrive/'Colab Notebooks'/research/guidedresearchproject-rongo1

# %ls ../data/vegetable_images

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/research/guidedresearchproject-rongo1


In [2]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


data_dir = '../data/vegetable_images'


transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


train_dataset = ImageFolder(root=data_dir + '/train', transform=transform)
val_dataset = ImageFolder(root=data_dir + '/validation', transform=transform)

val_dataset_with_noise = ImageFolder(root=data_dir + '/gaussian_noise/validation/100', transform=transform)
test_dataset = ImageFolder(root=data_dir + '/gaussian_noise/test/100', transform=transform)
test_dataset_without_noise = ImageFolder(root=data_dir + '/test', transform=transform)


dataloaders = {'train':  DataLoader(train_dataset, batch_size=64, shuffle=True),
               'val': DataLoader(val_dataset, batch_size=64)
               }

dataloaders_with_noise = {'train':  DataLoader(train_dataset, batch_size=64, shuffle=True),
               'val': DataLoader(val_dataset_with_noise, batch_size=64)
               }


test_loader = DataLoader(test_dataset, batch_size=64)
test_loader_without_noise = DataLoader(test_dataset_without_noise, batch_size=64)




FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/vegetable_images/train'

In [16]:
print(len(dataloaders['train'].dataset))
print(len(dataloaders['val'].dataset))

dataset_sizes = {'train' : len(dataloaders['train'].dataset), 'val': len(dataloaders['val'].dataset), 'test': len(test_loader.dataset)}
print(len(test_loader.dataset))
classes  = train_dataset.classes
classes

15000
3000
3000


['Bean',
 'Bitter_Gourd',
 'Bottle_Gourd',
 'Brinjal',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Cucumber',
 'Papaya',
 'Potato',
 'Pumpkin',
 'Radish',
 'Tomato']

In [17]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [18]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 199MB/s] 


In [19]:
num_features = model.fc.in_features


model.fc = torch.nn.Linear(num_features, len(classes))

In [20]:
model = model.to(device)

In [21]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader_without_noise:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(len(images)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

KeyboardInterrupt: ignored

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)


        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0


            for inputs, labels in dataloaders[phase]:
                inputs =  torch.tensor(inputs).to(device)
                labels = torch.tensor(labels).to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()


                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


    model.load_state_dict(best_model_wts)
    return model

Accuracy of the network: 7.1 %
Accuracy of Bean: 1.0 %
Accuracy of Bitter_Gourd: 10.0 %
Accuracy of Bottle_Gourd: 10.0 %
Accuracy of Brinjal: 13.0 %
Accuracy of Broccoli: 8.0 %
Accuracy of Cabbage: 9.5 %
Accuracy of Capsicum: 1.5 %
Accuracy of Carrot: 10.5 %
Accuracy of Cauliflower: 1.0 %
Accuracy of Cucumber: 7.0 %


In [ ]:

num_features = model.fc.in_features


new_fc = nn.Linear(num_features, len(classes))


model.fc = nn.Sequential(
    nn.Dropout(0.5),
    new_fc,
    nn.Softmax(dim=1)  )